# opt_einsum

Let's explore the very basics of `opt_einsum`.

In [19]:
from functools import reduce

import numpy as np
import opt_einsum as oe

In [6]:
SEED = 42
N_SITES = 100
BOND_MIN = 5
BOND_MAX = 100
PHYS_DIM = 8

In [3]:
rng = np.random.default_rng(SEED)

### Random tensor network

First we create a simple random tensor network, a Matrix Product State (MPS) with 100 sites. The bond dimensions are set at random between 5 and 100.

In [10]:
bdims = np.concatenate([rng.integers(low=BOND_MIN, high=BOND_MAX, size=N_SITES-1), np.array([1])])

tn = [
    rng.uniform(low=-1.0, high=1.0, size=(bdims[k-1], PHYS_DIM, bdims[k])) 
    for k in range(N_SITES)
]

## Computing the norm

The example computation here, is the calculation of the norm of the MPS:

<img src="img/norm.png" width=500px>

First we construct the `einsum` string, this is a bit tedious...

In [11]:
phys_labels = [oe.get_symbol(k) for k in range(N_SITES)]
top_labels = [
    "{}{}{}".format(oe.get_symbol(k), phys_labels[k-N_SITES], oe.get_symbol(k+1))
    for k in range(N_SITES, 2*N_SITES)
]
bottom_labels = [
    "{}{}{}".format(oe.get_symbol(k), phys_labels[k-2*N_SITES], oe.get_symbol(k+1))
    for k in range(2*N_SITES, 3*N_SITES)
]

einsum_str = ",".join(top_labels + bottom_labels)

In case, you wonder how that looks, here it is:

In [12]:
print(einsum_str)

ðañ,ñbò,òcó,ódô,ôeõ,õfö,ög÷,÷hø,øiù,ùjú,úkû,ûlü,ümý,ýnþ,þoÿ,ÿpĀ,Āqā,ārĂ,Ăsă,ătĄ,Ąuą,ąvĆ,Ćwć,ćxĈ,Ĉyĉ,ĉzĊ,ĊAċ,ċBČ,ČCč,čDĎ,ĎEď,ďFĐ,ĐGđ,đHĒ,ĒIē,ēJĔ,ĔKĕ,ĕLĖ,ĖMė,ėNĘ,ĘOę,ęPĚ,ĚQě,ěRĜ,ĜSĝ,ĝTĞ,ĞUğ,ğVĠ,ĠWġ,ġXĢ,ĢYģ,ģZĤ,ĤÀĥ,ĥÁĦ,ĦÂħ,ħÃĨ,ĨÄĩ,ĩÅĪ,ĪÆī,īÇĬ,ĬÈĭ,ĭÉĮ,ĮÊį,įËİ,İÌı,ıÍĲ,ĲÎĳ,ĳÏĴ,ĴÐĵ,ĵÑĶ,ĶÒķ,ķÓĸ,ĸÔĹ,ĹÕĺ,ĺÖĻ,Ļ×ļ,ļØĽ,ĽÙľ,ľÚĿ,ĿÛŀ,ŀÜŁ,ŁÝł,łÞŃ,Ńßń,ńàŅ,Ņáņ,ņâŇ,Ňãň,ňäŉ,ŉåŊ,Ŋæŋ,ŋçŌ,Ōèō,ōéŎ,Ŏêŏ,ŏëŐ,Őìő,őíŒ,Œîœ,œïŔ,Ŕaŕ,ŕbŖ,Ŗcŗ,ŗdŘ,Řeř,řfŚ,Śgś,śhŜ,Ŝiŝ,ŝjŞ,Şkş,şlŠ,Šmš,šnŢ,Ţoţ,ţpŤ,Ťqť,ťrŦ,Ŧsŧ,ŧtŨ,Ũuũ,ũvŪ,Ūwū,ūxŬ,Ŭyŭ,ŭzŮ,ŮAů,ůBŰ,ŰCű,űDŲ,ŲEų,ųFŴ,ŴGŵ,ŵHŶ,ŶIŷ,ŷJŸ,ŸKŹ,ŹLź,źMŻ,ŻNż,żOŽ,ŽPž,žQſ,ſRƀ,ƀSƁ,ƁTƂ,ƂUƃ,ƃVƄ,ƄWƅ,ƅXƆ,ƆYƇ,ƇZƈ,ƈÀƉ,ƉÁƊ,ƊÂƋ,ƋÃƌ,ƌÄƍ,ƍÅƎ,ƎÆƏ,ƏÇƐ,ƐÈƑ,ƑÉƒ,ƒÊƓ,ƓËƔ,ƔÌƕ,ƕÍƖ,ƖÎƗ,ƗÏƘ,ƘÐƙ,ƙÑƚ,ƚÒƛ,ƛÓƜ,ƜÔƝ,ƝÕƞ,ƞÖƟ,Ɵ×Ơ,ƠØơ,ơÙƢ,ƢÚƣ,ƣÛƤ,ƤÜƥ,ƥÝƦ,ƦÞƧ,Ƨßƨ,ƨàƩ,Ʃáƪ,ƪâƫ,ƫãƬ,Ƭäƭ,ƭåƮ,ƮæƯ,Ưçư,ưèƱ,ƱéƲ,ƲêƳ,Ƴëƴ,ƴìƵ,Ƶíƶ,ƶîƷ,ƷïƸ


## Contraction expressions

`opt_einsum` allows us to pre-optimize a contraction expression and reuse it, as long as all the shapes of the TN stay the same (this will come in handy later). First we collect the shapes:

In [13]:
shapes = [tensor.shape for tensor in tn]

`opt_einsum` selects the best optimization strategy, based on the number of operands in the expression. For an expression this large, it will default to `greedy`, which is a simple heuristic with linear scaling:

In [16]:
%%timeit 
oe.contract_expression(einsum_str, *shapes, *shapes, memory_limit=-1)

10.8 ms ± 275 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Let's see how long the actual contraction takes:

In [17]:
expr = oe.contract_expression(einsum_str, *shapes, *shapes, memory_limit=-1)

In [18]:
%%timeit
expr(*tn, *tn)

24.5 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


10.8ms to optimize the contraction path and 24.5ms to execute the actual contraction. If we want to reuse the expression, no additional optimization is necessary.

### A naive approach

`np.einsum` can't even handle (nearly) as many operands, hence for comparison we are limited to performing it in steps. We contract the following in each iteration of the loop:

<img src="img/con.png" width=400px>

In [23]:
%%timeit
res = np.einsum('abc,abd->cd', tn[0], tn[0])
for t in zip(tn[1:], tn[1:]):
   res = np.einsum('ab,acd,bce->de', res, t[0], t[1]) 

16.8 s ± 259 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


16.8s vs. 24.5ms, the naive approach takes about 685 times as long! Python loops aren't known to be particularly fast (although much improved in recent python versions). Let's try with `reduce`:

In [25]:
def contract(left, tensors):
    return np.einsum('ab,acd,bce->de', left, *tensors)

In [26]:
%%timeit
res = np.einsum('abc,abd->cd', tn[0], tn[0])
res = reduce(contract, zip(tn[1:], tn[1:]), res)

16.9 s ± 208 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That's about the same...

## More details on the contraction path

`opt_einsum` can tell us how much work it avoids in theory by optimizing the contraction path:

In [31]:
path_info = oe.contract_path(einsum_str, *tn, *tn)
print(path_info[1])

  Complete contraction:  ðañ,ñbò,òcó,ódô,ôeõ,õfö,ög÷,÷hø,øiù,ùjú,úkû,ûlü,ümý,ýnþ,þoÿ,ÿpĀ,Āqā,ārĂ,Ăsă,ătĄ,Ąuą,ąvĆ,Ćwć,ćxĈ,Ĉyĉ,ĉzĊ,ĊAċ,ċBČ,ČCč,čDĎ,ĎEď,ďFĐ,ĐGđ,đHĒ,ĒIē,ēJĔ,ĔKĕ,ĕLĖ,ĖMė,ėNĘ,ĘOę,ęPĚ,ĚQě,ěRĜ,ĜSĝ,ĝTĞ,ĞUğ,ğVĠ,ĠWġ,ġXĢ,ĢYģ,ģZĤ,ĤÀĥ,ĥÁĦ,ĦÂħ,ħÃĨ,ĨÄĩ,ĩÅĪ,ĪÆī,īÇĬ,ĬÈĭ,ĭÉĮ,ĮÊį,įËİ,İÌı,ıÍĲ,ĲÎĳ,ĳÏĴ,ĴÐĵ,ĵÑĶ,ĶÒķ,ķÓĸ,ĸÔĹ,ĹÕĺ,ĺÖĻ,Ļ×ļ,ļØĽ,ĽÙľ,ľÚĿ,ĿÛŀ,ŀÜŁ,ŁÝł,łÞŃ,Ńßń,ńàŅ,Ņáņ,ņâŇ,Ňãň,ňäŉ,ŉåŊ,Ŋæŋ,ŋçŌ,Ōèō,ōéŎ,Ŏêŏ,ŏëŐ,Őìő,őíŒ,Œîœ,œïŔ,Ŕaŕ,ŕbŖ,Ŗcŗ,ŗdŘ,Řeř,řfŚ,Śgś,śhŜ,Ŝiŝ,ŝjŞ,Şkş,şlŠ,Šmš,šnŢ,Ţoţ,ţpŤ,Ťqť,ťrŦ,Ŧsŧ,ŧtŨ,Ũuũ,ũvŪ,Ūwū,ūxŬ,Ŭyŭ,ŭzŮ,ŮAů,ůBŰ,ŰCű,űDŲ,ŲEų,ųFŴ,ŴGŵ,ŵHŶ,ŶIŷ,ŷJŸ,ŸKŹ,ŹLź,źMŻ,ŻNż,żOŽ,ŽPž,žQſ,ſRƀ,ƀSƁ,ƁTƂ,ƂUƃ,ƃVƄ,ƄWƅ,ƅXƆ,ƆYƇ,ƇZƈ,ƈÀƉ,ƉÁƊ,ƊÂƋ,ƋÃƌ,ƌÄƍ,ƍÅƎ,ƎÆƏ,ƏÇƐ,ƐÈƑ,ƑÉƒ,ƒÊƓ,ƓËƔ,ƔÌƕ,ƕÍƖ,ƖÎƗ,ƗÏƘ,ƘÐƙ,ƙÑƚ,ƚÒƛ,ƛÓƜ,ƜÔƝ,ƝÕƞ,ƞÖƟ,Ɵ×Ơ,ƠØơ,ơÙƢ,ƢÚƣ,ƣÛƤ,ƤÜƥ,ƥÝƦ,ƦÞƧ,Ƨßƨ,ƨàƩ,Ʃáƪ,ƪâƫ,ƫãƬ,Ƭäƭ,ƭåƮ,ƮæƯ,Ưçư,ưèƱ,ƱéƲ,ƲêƳ,Ƴëƴ,ƴìƵ,Ƶíƶ,ƶîƷ,ƷïƸ->ðƸ
         Naive scaling:  301
     Optimized scaling:  7
      Naive FLOP count:  1.810e+417
  Optimized FLOP count:  6.510e+8
   Theoretical speedup:  2.781e+408
  Largest